**Outline**

1. Append the datasets together, make sure the columns line up
2. Clean them.
3. Run test to see if the likelihood of winnning first serve before you hit it is higher than that of the second serve.
    - if so, players should just use their first serve (mixture) for their second serve.
        - mixture is defined as the set of speed, placement, and spin that spans the first serve.
            - spin is unobserved (I think)
                -Generally, you can guess that wide deuce serves are slice and wide ad serves are kick (topspin)
    - what is the psychological impact of double faulting? Does this justify being more conservative with your second serve?

**Other goals**
1. See what the optimum speed of serve to hit is in terms of likelihood to win at time of striking the ball
    - Unforturnately, I dont know the speed of serves that miss.
        - Would I need to do a stepwise regression here? Or is it instrumental variables?
            - looking to find another dataset that has shows the effect serve speed has on liklihood of making your serve and integrate it into the analysis.
2. Is there an interactive term? Do past serve types (as defined by placement, speed) determine how effective later serves are?
    - Could create some index of diversity of serves, see if it's predictive of average serve success of a match
    - Are more rare serve types (defined by speed/placement) more effective? 
3. Are certain serve types more effective in big moments?
4. Should I try regressions if I finish early?
    - could regress serve speed/effectiveness on number of points
        - people tend to get more tired as the match goes on
    - Could create a more complete logit model.
5. See if there is a difference in the men's and women's data
6. Heatmap of court/serving box separated into wide, center, T
7. Could interpolate point number
8. Take the average serve win percentage and average make percentage by player in a match, and compare those.

break out later serves into own DF and compare corr coefficient and compare with earleir serves

know what successful serves look like, use clustering. kNow speed and direction, plot that data, and for the color, plot whether the serve was successful or not. You might see some separation in taht 2d plot and see the clusters. can see the average and see what gave that cluster it's 


Codebook:

PointServer: Numeric indicator of player who is serving

PointWinner: Numeric indicator of player who won the point

Set: Numeric of set number

Game: Numeric of game number

Time: Elapsed time at start of point (hh:mm:ss)

Ace: Numeric indicator if server made and ace

Winner: Numeric indicator if point ended with a winner

DoubleFault: Numeric indicator if point ended in double fault

UnforcedError: Numeric indicator if point ended in an unforced error

NetPoint: Numeric indicator if point was a net point

WinnerShotType: Character of type of shot for winner (F or B)

ServerError: Logical if server made an error

ReceiverError: Logical if receiver made an error

SpeedMPH: Numeric of MPH of serve

ServeNumber: Numeric indicator of first or second serve

BreakPointWon: Numeric indicator if break point was won

BreakPointOpportunity: Numeric indicator if had a break point opportunity

GameWinner: Numeric if point ended in game winner and for which player

ForcedError: Numeric of whether point ended in forced error

MatchCode: Identifier of the match

Round: Numeric value of round (1 = First, 7 = Final)

ServerName: Character of server's name

Player1: Character of Player 1 name

Player2: Character of PLayer 2 name

In [29]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

# from statsmodels.stats.multitests import multipletests

%matplotlib inline

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [141]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading in the dataframes, combining them for analysis

In [ ]:
# write a loop to load the different data files. Format: year-openname-points
slams = ["usopen", "ausopen", "wimbledon", "frenchopen"]
years = range(2011, 2019)
yearslams = []
df = pd.DataFrame()
for year in years:
    for slam in slams:
        yearslam = f"{year}-{slam}"
        yearslams = yearslams + [yearslam]
        data_path = f"data/tennis_slam_pointbypoint-master/{yearslam}-points.csv"
        df_new = pd.read_csv(data_path, encoding="ISO-8859-1")
        df_new["year"] = year
        df_new["slam"] = slam
        df = df.append(df_new, ignore_index=True)
aussy2020 = pd.read_csv(
    "data/tennis_slam_pointbypoint-master/2020-ausopen-points.csv",
    index_col=0,
    encoding="ISO-8859-1",
)
aussy2020["year"] = 2020
aussy2020["slam"] = "ausopen"
df = df.append(aussy2020, ignore_index=True)
print("done reading")

Adding year, and tournament columns, then creating a multiindex by match no and point no to uniquely identify each point.

In [63]:
# trying to create year and tournament columns without dropping null match ids, but only about 5% are null
#df["year"] = df["match_id"].apply(lambda x: x.split("-")[0] if x != None else None)
#df = df.assign(year=np.where(df.match_id.notnull(), df.match_id.str.split('-')[0], None))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Come back to the below, trying to make a year and tournament field 

(UPDATE: did that above)

In [122]:
df1 = df
df1["match_id1"] = df1["match_id"]
df1["PointNumber1"] = df1["PointNumber"]

#I should interpolate point number before this
df1=df1.dropna(subset=['match_id', 'PointNumber'])
df1 = df1.set_index(["match_id", "PointNumber"]).sort_index()
#df1=df1[~df1.index.duplicated(keep='first')]
#df1['year'] = None
#df1['tournament']= None
#df1.loc[df1['match_id1'].str.find('-'), 'year']=df1["match_id1"].str.split("-")[0]
#df1.loc[df1['match_id1'].str.find('-'), 'year']=df1["match_id1"].str.split("-")[1]

#df["tournament"] = df["match_id"].str.split("-")[1]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [139]:
df1.notnull().sum().sort_values()

ServingTo               27309
Serve_Direction         83016
Winner_FH               83016
Winner_BH               83016
P2FirstSrvWon          114538
P2FirstSrvIn           114538
P1FirstSrvIn           114538
P1SecondSrvIn          114538
P2SecondSrvIn          114538
P1SecondSrvWon         114538
P2SecondSrvWon         114538
P1ForcedError          114538
P2ForcedError          114538
Rally                  114538
P1FirstSrvWon          114538
P2TurningPoint         123608
P1TurningPoint         123614
ReturnDepth            164116
ServeDepth             202328
ServeWidth             202328
RallyCount             428970
P2Momentum             453529
P1Momentum             453529
P1DistanceRun          492682
P2DistanceRun          492682
WinnerType             656195
WinnerShotType         656195
ServeNumber            749151
History               1031614
P1GamesWon            1035814
P2GamesWon            1035814
P1PointsWon           1035814
P2PointsWon           1035814
P1BreakPoi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [153]:
# delete later
df1.loc[("2011-ausopen-1101"), "Speed_MPH"].mean()

90.40322580645162

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [143]:
df1.head(15)

ElapsedTime  SetNo  P1GamesWon  P2GamesWon  \
match_id          PointNumber                                              
2011-ausopen-1101 0              00:00:00    1.0         0.0         0.0   
                  1               0:00:00    1.0         0.0         0.0   
                  2               0:00:23    1.0         0.0         0.0   
                  3               0:00:59    1.0         0.0         0.0   
                  4               0:01:57    1.0         1.0         0.0   
                  5               0:03:23    1.0         1.0         0.0   
                  6               0:04:04    1.0         1.0         0.0   
                  7               0:04:27    1.0         1.0         0.0   
                  8               0:05:01    1.0         1.0         0.0   
                  9               0:05:21    1.0         1.0         0.0   
                  10              0:05:53    1.0         2.0         0.0   
                  11              0:06:39    1.0         2.0         0.0   
                  12              0:07:10    1.0         2.0         0.0   
                  13              0:07:39    1.0         2.0         0.0   
                  14              0:08:10    1.0         2.0         0.0   

                               SetWinner  GameNo  GameWinner  PointWinner  \
match_id          PointNumber                                               
2011-ausopen-1101 0                  0.0     1.0         0.0          0.0   
                  1                  0.0     1.0         0.0          1.0   
                  2                  0.0     1.0         0.0          1.0   
                  3                  0.0     1.0         0.0          1.0   
                  4                  0.0     1.0         1.0          1.0   
                  5                  0.0     2.0         0.0          1.0   
                  6                  0.0     2.0         0.0          1.0   
                  7                  0.0     2.0         0.0          2.0   
                  8                  0.0     2.0         0.0          1.0   
                  9                  0.0     2.0         0.0          2.0   
                  10                 0.0     2.0         1.0          1.0   
                  11                 0.0     3.0         0.0          2.0   
                  12                 0.0     3.0         0.0          1.0   
                  13                 0.0     3.0         0.0          1.0   
                  14                 0.0     3.0         0.0          1.0   

                               PointServer  Speed_KMH  Rally P1Score P2Score  \
match_id          PointNumber                                                  
2011-ausopen-1101 0                    0.0        0.0    0.0       0       0   
                  1                    2.0      166.0    4.0      15       0   
                  2                    2.0      132.0    4.0      30       0   
                  3                    2.0      144.0    8.0      40       0   
                  4                    2.0      137.0    6.0       0       0   
                  5                    1.0      149.0    5.0      15       0   
                  6                    1.0      201.0    1.0      30       0   
                  7                    1.0        0.0    0.0      30      15   
                  8                    1.0      171.0    1.0      40      15   
                  9                    1.0      200.0    2.0      40      30   
                  10                   1.0      145.0    3.0       0       0   
                  11                   2.0      158.0    1.0       0      15   
                  12                   2.0      162.0    4.0      15      15   
                  13                   2.0      161.0    6.0      30      15   
                  14                   2.0      172.0    6.0      40      15   

                               P1Momentum  P2Momentum  P1PointsWon

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [124]:
serve = df1[
    [
        "PointWinner",
        "PointServer",
        "Speed_MPH",
        "P1PointsWon",
        "P2PointsWon",
        "P1Ace",
        "P2Ace",
        "ServeIndicator",
        "Serve_Direction",
        "ServeNumber",
        "RallyCount",
        "ServeWidth",
        "ServeDepth",
        "match_id1",
        "PointNumber1",
    ]
]
serve.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1129523 entries, ('2011-ausopen-1101', 0) to ('2018-wimbledon-2701', '99')
Data columns (total 15 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   PointWinner      1128883 non-null  float64
 1   PointServer      1128883 non-null  float64
 2   Speed_MPH        1081343 non-null  float64
 3   P1PointsWon      1035814 non-null  float64
 4   P2PointsWon      1035814 non-null  float64
 5   P1Ace            1129521 non-null  float64
 6   P2Ace            1129521 non-null  float64
 7   ServeIndicator   1036454 non-null  float64
 8   Serve_Direction  83016 non-null    float64
 9   ServeNumber      749151 non-null   float64
 10  RallyCount       428970 non-null   float64
 11  ServeWidth       202328 non-null   object 
 12  ServeDepth       202328 non-null   object 
 13  match_id1        1129523 non-null  object 
 14  PointNumber1     1129523 non-null  object 
dtypes: float64(11), objec

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [125]:
serve.head(10)

PointWinner  PointServer  Speed_MPH  \
match_id          PointNumber                                        
2011-ausopen-1101 0                    0.0          0.0        0.0   
                  1                    1.0          2.0      103.0   
                  2                    1.0          2.0       82.0   
                  3                    1.0          2.0       89.0   
                  4                    1.0          2.0       85.0   
                  5                    1.0          1.0       92.0   
                  6                    1.0          1.0      124.0   
                  7                    2.0          1.0        0.0   
                  8                    1.0          1.0      106.0   
                  9                    2.0          1.0      124.0   

                               P1PointsWon  P2PointsWon  P1Ace  P2Ace  \
match_id          PointNumber                                           
2011-ausopen-1101 0                    0.0          0.0    0.0    0.0   
                  1                    1.0          0.0    0.0    0.0   
                  2                    2.0          0.0    0.0    0.0   
                  3                    3.0          0.0    0.0    0.0   
                  4                    4.0          0.0    0.0    0.0   
                  5                    5.0          0.0    0.0    0.0   
                  6                    6.0          0.0    0.0    0.0   
                  7                    6.0          1.0    0.0    0.0   
                  8                    7.0          1.0    1.0    0.0   
                  9                    7.0          2.0    0.0    0.0   

                               ServeIndicator  Serve_Direction  ServeNumber  \
match_id          PointNumber                                                 
2011-ausopen-1101 0                       0.0              NaN          NaN   
                  1                       2.0              NaN          NaN   
                  2                       2.0              NaN          NaN   
                  3                       2.0              NaN          NaN   
                  4                       1.0              NaN          NaN   
                  5                       1.0              NaN          NaN   
                  6                       1.0              NaN          NaN   
                  7                       1.0              NaN          NaN   
                  8                       1.0              NaN          NaN   
                  9                       1.0              NaN          NaN   

                               RallyCount ServeWidth ServeDepth  \
match_id          PointNumber                                     
2011-ausopen-1101 0                   NaN        NaN        NaN   
                  1                   NaN        NaN        NaN   
                  2                   NaN        NaN        NaN   
                  3                   NaN        NaN        NaN   
                  4                   NaN        NaN        NaN   
                  5                   NaN        NaN        NaN   
                  6                   NaN        NaN        NaN   
                  7                   NaN        NaN        NaN   
                  8                   NaN        NaN        NaN   
                  9                   NaN        NaN        NaN   

                                       match_id1 PointNumber1  
match_id          PointNumber                                  
2011-ausopen-1101 0            2011-ausopen-1101            0  
                  1            2011-ausopen-1101            1  
                  2            2011-ausopen-1101            2  
                  3            2011-ausopen-1101            3  
                  4            2011-ausopen-1101            4  
                  5            2011-ausopen-1101            5  
                  6            2011-ausopen-110

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [126]:
serve.shape

(1129523, 15)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [127]:
serve.isna().sum()

PointWinner            640
PointServer            640
Speed_MPH            48180
P1PointsWon          93709
P2PointsWon          93709
P1Ace                    2
P2Ace                    2
ServeIndicator       93069
Serve_Direction    1046507
ServeNumber         380372
RallyCount          700553
ServeWidth          927195
ServeDepth          927195
match_id1                0
PointNumber1             0
dtype: int64

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [128]:
# make a dataset without serve_direction, so we can drop all null rows and clean it a bit
serve_clean = df1[
    [
        "PointWinner",
        "PointServer",
        "Speed_MPH",
        "P1PointsWon",
        "P2PointsWon",
        "P1Ace",
        "P2Ace",
        "ServeIndicator",
        "ServeNumber",
        "RallyCount",
        "ServeWidth",
        "ServeDepth",
        "match_id1",
        "PointNumber1",
    ]
]
serve_clean = serve_clean.dropna()
serve_clean = serve_clean[serve_clean["Speed_MPH"] != 0]
serve_clean['ServerWin']= 1
serve_clean.loc[serve_clean["PointServer"] != serve_clean['PointWinner'], "ServerWin"] = 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [129]:
serve_clean.size

2919255

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We still have more than 1 million rows, good enough as far as data size, but I should be wary of biasing the data toward matches that are later in the tournament, therefore having higher quality players

Make a dataset of only the specific variables related to speed

In [130]:
speed_vars = serve_clean[
    [
        "ServeNumber",
        "Speed_MPH",
        "PointWinner",
        "PointServer",
        "ServeWidth",
        "ServeDepth",
        "ServerWin",
    ]
]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [131]:
speed_vars["PointServer"].value_counts()

2.0    97907
1.0    96710
Name: PointServer, dtype: int64

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [132]:
speed_first = speed_vars[speed_vars["ServeNumber"] == 1]
speed_second = speed_vars[speed_vars["ServeNumber"] == 2]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [133]:
print('First serve win percentage: ')
print(speed_first['ServerWin'].mean())
print('Second serve win percentage: ')
print(speed_second['ServerWin'].mean())


First serve win percentage: 
0.6960487773815588
Second serve win percentage: 
0.5517431325650504


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Just trying out grouping by server

In [167]:
speed_vars.loc[("2016-frenchopen-1101"), ["ServerWin", "PointServer"]].groupby(
    by=["PointServer"]
).mean()

,ServerWin
PointServer,
1.0,0.754098
2.0,0.513889


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

why can't I sort by pointserver?

In [175]:
#speed_vars.PointServer = speed_vars.PointServer.astype(int)
#speed_vars["ServerWin"].groupby(by=["match_id", "PointServer"]).mean()

/Users/jamesmoss/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


KeyError: 'PointServer'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [156]:
speed_vars.head()

ServeNumber  Speed_MPH  PointWinner  \
match_id             PointNumber                                        
2016-frenchopen-1101 2                    2.0       85.0          1.0   
                     3                    1.0      118.0          2.0   
                     4                    2.0       94.0          2.0   
                     5                    1.0      113.0          2.0   
                     6                    2.0       80.0          1.0   

                                  PointServer ServeWidth ServeDepth  ServerWin  
match_id             PointNumber                                                
2016-frenchopen-1101 2                    2.0         BW       NCTL          0  
                     3                    2.0          C       NCTL          1  
                     4                    2.0         BW       NCTL          1  
                     5                    2.0          W       NCTL          1  
                     6                    1.0          C       NCTL          1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [170]:
speed_vars.groupby("match_id").size()

match_id
2016-frenchopen-1101    133
2016-frenchopen-1109    118
2016-frenchopen-1117     82
2016-frenchopen-1132    165
2016-frenchopen-1141    189
                       ... 
2018-wimbledon-2503     177
2018-wimbledon-2504     144
2018-wimbledon-2601     122
2018-wimbledon-2602      94
2018-wimbledon-2701      98
Length: 1177, dtype: int64

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>